In [ ]:
from ast import literal_eval
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
df = pd.read_csv(
    "/Users/202837/Documents/Python/Food Price API/newRecipes.csv", index_col=[0]
)

In [ ]:
def get_prices(ing: str) -> list[float]:
    """Get the prices for each item"""
    ingList = "\\n".join(literal_eval(ing))

    widget = requests.post(
        "https://api.spoonacular.com/recipes/visualizePriceEstimator",
        data={
            "apiKey": "17a880d8a40f4451bccdc80f4d3aa38e",
            "ingredientList": ingList,
            "servings": "1",
            "defaultCss": "false",
        },
    )
    soup = BeautifulSoup(widget.text, "html.parser")
    things = soup.select_one("#spoonacularPriceBreakdownTable").find_all("div")
    prices = repr(things[3]).split("<br/>")[1:-1]

    return prices

In [ ]:
def get_nutrs(ing):
    """Return the nutrition values for a list of items"""
    ingList = "\\n".join(literal_eval(ing))
    widget = requests.post(
        "https://api.spoonacular.com/recipes/visualizeNutrition",
        data={
            "apiKey": "17a880d8a40f4451bccdc80f4d3aa38e",
            "ingredientList": ingList,
            "servings": "1",
            "defaultCss": "false",
        },
    )

    soup = BeautifulSoup(widget.text, "html.parser")

    quickview = soup.find("div", attrs={"itemprop": "nutrition"})
    facts = [element.get_text() for element in quickview][1:]
    return {
        "_".join(stuff[1].split(" ")).lower(): stuff[0]
        for fact in facts
        if (stuff := fact.split(" ", 1))
    }

In [ ]:
def sum_price(prices):
    """Return the total price for a list of prices"""
    pricesList = [float(price[1:]) for price in prices]
    totalPrice = sum(pricesList)
    return round(totalPrice, 2)

In [ ]:
amount = 10

ingredientPrices = df.Ingredients.iloc[:amount].apply(get_prices)
ingredientNutrients = df.Ingredients.iloc[:amount].apply(get_nutrs)
price = ingredientPrices.apply(sum_price)

ingredientPrices.name = "Ing. Prices"
ingredientNutrients.name = "Ing. Nutr"
price.name = "Price"

In [ ]:
outputDf = df.iloc[:amount].join(ingredientPrices).join(price).join(ingredientNutrients)
outputDf # 